In [35]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np

import pyodbc
import urllib
import sqlite3

### Store CSV into DataFrame

In [14]:
#Pulls in Jobs Data
csv_file = "monster_com-job_sample.csv"
jobs_df = pd.read_csv(csv_file)
jobs_df.head()


,country,country_code,date_added,has_expired,job_board,job_description,job_title,job_type,location,organization,page_url,salary,sector,uniq_id
0,United States of America,US,NaN,No,jobs.monster.com,TeamSoft is seeing an IT Support Specialist to...,IT Support Technician Job in Madison,Full Time Employee,"Madison, WI 53702",NaN,http://jobview.monster.com/it-support-technici...,NaN,IT/Software Development,11d599f229a80023d2f40e7c52cd941e
1,United States of America,US,NaN,No,jobs.monster.com,The Wisconsin State Journal is seeking a flexi...,Business Reporter/Editor Job in Madison,Full Time,"Madison, WI 53708",Printing and Publishing,http://jobview.monster.com/business-reporter-e...,NaN,NaN,e4cbb126dabf22159aff90223243ff2a
2,United States of America,US,NaN,No,jobs.monster.com,Report this job About the Job DePuy Synthes Co...,Johnson & Johnson Family of Companies Job Appl...,"Full Time, Employee",DePuy Synthes Companies is a member of Johnson...,Personal and Household Services,http://jobview.monster.com/senior-training-lea...,NaN,NaN,839106b353877fa3d896ffb9c1fe01c0
3,United States of America,US,NaN,No,jobs.monster.com,Why Join Altec? If you’re considering a career...,Engineer - Quality Job in Dixon,Full Time,"Dixon, CA",Altec Industries,http://jobview.monster.com/engineer-quality-jo...,NaN,Experienced (Non-Manager),58435fcab804439efdcaa7ecca0fd783
4,United States of America,US,NaN,No,jobs.monster.com,Position ID# 76162 # Positions 1 State CT C...,Shift Supervisor - Part-Time Job in Camphill,Full Time Employee,"Camphill, PA",Retail,http://jobview.monster.com/shift-supervisor-pa...,NaN,Project/Program Management,64d0272dc8496abfd9523a8df63c184c


### Create new data with select columns

In [15]:
#Copy Columns of Interest.
new_jobs_df = jobs_df[['job_title', 'job_type', 'sector','location']].copy()
new_jobs_df.head()

,job_title,job_type,sector,location
0,IT Support Technician Job in Madison,Full Time Employee,IT/Software Development,"Madison, WI 53702"
1,Business Reporter/Editor Job in Madison,Full Time,NaN,"Madison, WI 53708"
2,Johnson & Johnson Family of Companies Job Appl...,"Full Time, Employee",NaN,DePuy Synthes Companies is a member of Johnson...
3,Engineer - Quality Job in Dixon,Full Time,Experienced (Non-Manager),"Dixon, CA"
4,Shift Supervisor - Part-Time Job in Camphill,Full Time Employee,Project/Program Management,"Camphill, PA"


In [16]:
#Renames job types.
new_jobs_df['job_type'] = new_jobs_df['job_type'].map(
    lambda x: x if pd.isnull(x) else 'Full Time' if 'Full' in x else 'Part Time' if 'Part' in x else 'Other'
)

new_jobs_df.head()

,job_title,job_type,sector,location
0,IT Support Technician Job in Madison,Full Time,IT/Software Development,"Madison, WI 53702"
1,Business Reporter/Editor Job in Madison,Full Time,NaN,"Madison, WI 53708"
2,Johnson & Johnson Family of Companies Job Appl...,Full Time,NaN,DePuy Synthes Companies is a member of Johnson...
3,Engineer - Quality Job in Dixon,Full Time,Experienced (Non-Manager),"Dixon, CA"
4,Shift Supervisor - Part-Time Job in Camphill,Full Time,Project/Program Management,"Camphill, PA"


In [17]:

#Parsing through the location column and cleaning it up
new_jobs_df=new_jobs_df[new_jobs_df['location'].str.len() < 30]

location=new_jobs_df['location'].str.split(',')
new_jobs_df['City']=location.str[0]
new_jobs_df['State']=location.str[1].str[0:3]


new_jobs_df.reset_index(inplace=True, drop=True)


In [18]:
#Drops the rows with NAs in the State C
new_jobs_df.dropna(subset=['State'],inplace=True)

new_jobs_df.drop(['location'],axis=1,inplace=True)

In [19]:
new_jobs_df.head()

,job_title,job_type,sector,City,State
0,IT Support Technician Job in Madison,Full Time,IT/Software Development,Madison,WI
1,Business Reporter/Editor Job in Madison,Full Time,NaN,Madison,WI
2,Engineer - Quality Job in Dixon,Full Time,Experienced (Non-Manager),Dixon,CA
3,Shift Supervisor - Part-Time Job in Camphill,Full Time,Project/Program Management,Camphill,PA
4,Construction PM - Charlottesville Job in Charl...,Full Time,Experienced (Non-Manager),Charlottesville,VA
6,Mailroom Clerk Job in Austin,Full Time,Experienced (Non-Manager),Austin,TX
7,Housekeeper Job in Austin,Part Time,Customer Support/Client Care,Austin,TX
8,Video Data Management /Transportation Technici...,NaN,NaN,Chesterfield,MO
9,Aflac Insurance Sales Agent Job in Berryville,Full Time,Customer Support/Client Care,Berryville,VA
10,Sales Associate Job in Columbus,Full Time,Customer Support/Client Care,Columbus,IN


In [37]:

#params=urllib.parse.quote_plus('Driver={ODBC Driver 13 for SQL Server};Server=tcp:etlproject.database.windows.net,1433;Database=ETLProject;Uid=datascience@etlproject;Pwd=Abcde12345;Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;')

#engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

#sqlite3.connect('etlproject.sqlite')

#engine = create_engine("sqlite:///etlproject.sqlite")


In [41]:
new_jobs_df.to_csv('Cleaned_jobs.csv')

,job_title,job_type,sector,City,State
0,IT Support Technician Job in Madison,Full Time,IT/Software Development,Madison,WI
1,Business Reporter/Editor Job in Madison,Full Time,None,Madison,WI
2,Engineer - Quality Job in Dixon,Full Time,Experienced (Non-Manager),Dixon,CA
3,Shift Supervisor - Part-Time Job in Camphill,Full Time,Project/Program Management,Camphill,PA
4,Construction PM - Charlottesville Job in Charl...,Full Time,Experienced (Non-Manager),Charlottesville,VA
